# TII Falcon 7B Tutorial in HugginFace Transformers and Langchain 
In this tutorial I'm going to show how we can use the open-source Falcon 7B model developed by Technology Innovation Institute TII. Falcon supposedly outperforms GPT-3 for only 75% of the training compute budget whilst requiring a fifth of the compute at inference time. 


1. Installing packages

In [1]:
!pip install -q transformers 
!pip install -q accelerate einops 
!pip install -q langchain 
!pip install -q xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.6/934.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Let's initialize a HuggingFace pipeline and load the LLM

If you're using Colab, make sure you use the GPU runtime, preferably the Pro membership as you might need access to either an A100 or V100 GPU.  
Running this tutorial locally will not be sufficient for most personal setups.  
The general steps that will be taken:  
1. Loading the Falcon model from HuggingFace 
2. Loading the model tokenizer 

In [ ]:
from torch import cuda, bfloat16 
import transformers 
from transformers import AutoTokenizer, AutoModelForCausalLM 
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'  

#creating a model 
fmodel = AutoModelForCausalLM.from_pretrained(
    'tiiuae/falcon-7b-instruct',
    trust_remote_code=True,
    torch_dtype=bfloat16
)
fmodel.eval() 
fmodel.to(device) 
print(f'Model loaded on {device}') 

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The tokenizer is loaded from the tii model itself, as implemented below. 

In [3]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b-instruct')

Initializing the HF pipeline and setting up some additional parameters 

In [4]:
gen_text = transformers.pipeline(
    model=fmodel, 
    tokenizer=tokenizer, 
    task='text-generation', 
    return_full_text=True, 
    device=device, 
    max_length=10000, 
    temperature=0.1, 
    top_p=0.15, #select from top tokens whose probability adds up to 15%
    top_k=0, #selecting from top 0 tokens 
    repetition_penalty=1.1, #without a penalty, output starts to repeat 
    do_sample=True, 
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [6]:
result = gen_text("What is the name of the first president of the united arab emirates?") 
print(result[0]['generated_text']) 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


What is the name of the first president of the united arab emirates?
The name of the first president of the United Arab Emirates is Sheikh Zayed bin Sultan Al Nahyan.


### LangChain Implementation 

In [7]:
from langchain import PromptTemplate, LLMChain 
from langchain.chains.conversation.memory import ConversationBufferMemory,ConversationSummaryMemory
from langchain.llms import HuggingFacePipeline 
#Creating a basic prompt template 

template = """
I want you to act as an informative AI assistant. 
Answer the following question: {question}.
"""
prompt = PromptTemplate(
    input_variables=['question'], 
    template=template 
)

In [8]:
#showcasing the format of the prompt 
prompt.format(question="What is the name of the first US president? ")

'\nI want you to act as an informative AI assistant. \nAnswer the following question: What is the name of the first US president? .\n'

In [9]:
llm = HuggingFacePipeline(pipeline=gen_text)
llm_chain = LLMChain(llm=llm,prompt=prompt)

In [11]:
llm_chain.predict(question='List me some of the most important electrolytes') 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'Electrolytes are essential for maintaining the balance of fluids in the body. Some of the most important electrolytes include sodium, potassium, chloride, calcium, and magnesium.'

### Implementing LangChain conversational memory

In [13]:
#creating a new conversational template 
template = """You are an informative assistant chatting with a human. 
{chat_history}
Human:{human_input}
Assistant:""" 
#creating the prompt 
prompt = PromptTemplate(
    input_variables=["chat_history","human_input"], 
    template=template
) 
memory = ConversationBufferMemory(memory_key="chat_history") 

In [14]:
llm = HuggingFacePipeline(pipeline=gen_text) 
#adding memory to the llm chain 
llm_chain = LLMChain(llm=llm,prompt=prompt,memory=memory) 

In [15]:
#1st prompt 
llm_chain.predict(human_input='What is the first president of UAE?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


' The first president of UAE was Zayed bin Sultan Al Nahyan. He served as the President from 1971 until his death in 2006.\nUser '

In [16]:
#2nd prompt 
llm_chain.predict(human_input='Who was the president that came after him?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


' The current president of UAE is His Highness Sheikh Khalifa bin Zayed Al Nahyan. He has been in power since 2008.\nUser '